# Load data

## 현재 필요없는 libraries

In [1]:
import os
os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"

from rdkit.Chem import AllChem as Chem
from rdkit.Chem import QED
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem import RDConfig
# import numba as nb

## 필요 libraries

In [36]:
import pubchempy as pcp
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',200,'display.max_columns',50)
import csv
import time
import pickle
import os
import sys
import pubchempy as pcp
import Levenshtein as lvs

from tqdm import tqdm
sys.path.append(os.path.join(RDConfig.RDContribDir,'SA_Score'))
from sascorer import calculateScore
from sklearn.model_selection import train_test_split
from scipy import sparse as sps
from scipy.sparse import csr_matrix

In [11]:
root = "/home2/glee/Drug_Discovery_Research"
data_path = os.path.join(root, "data")
rawdata = pd.read_csv(os.path.join(data_path, "preprocessed/golden_set_20240520.csv"))

In [598]:
rawdata = rawdata_minor_tid_removed

In [666]:
rawdata_minor_tid_removed

,C_cid,C_seq_can_smiles,T_id,T_seq,IC50
0,1000,C1=CC=C(C=C1)C(CN)O,P03470,MNPNQKIITIGSICMVVGIISLILQIGNIISIWISHSIQTGNQNHT...,870000.0
1,10000015,CC(=O)NC1CCC2=CC3=C(C(=C2C4=CC=C(C(=O)C=C14)OC...,Q6B856,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...,2100.0
2,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P53708,MSPGASRGPRGSQAPLIAPLCCAAAALGMLLWSPACQAFNLDVEKL...,383.0
3,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P08648,MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDA...,17.0
4,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P05556,MNLQPIFWIGLISSVCCVFAQTDENRCLKANAKSCGECIQAGPNCG...,24.0
...,...,...,...,...,...
1779777,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P08253,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,120.0
1779778,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P14780,MSLWQPLVLVLLVLGCCFAAPRQRQSTLVLFPGDLRTNLTDRQLAE...,414.0
1779779,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P22894,MFSLKTLPFLLLLHVQISKAFPVSSKEKNTKTVQDYLEKFYQLPSN...,228.0
1779780,9999994,C1CN(CCC1CCC(=O)C2=CC3=C(C=C2)NCC3)CC4=CC(=CC=...,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,59.0


In [667]:
data_golden

,C_cid,C_seq_can_smiles,T_id,T_seq,IC50
0,1000,C1=CC=C(C=C1)C(CN)O,P03470,MNPNQKIITIGSICMVVGIISLILQIGNIISIWISHSIQTGNQNHT...,870000.0
1,10000015,CC(=O)NC1CCC2=CC3=C(C(=C2C4=CC=C(C(=O)C=C14)OC...,Q6B856,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...,2100.0
2,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P53708,MSPGASRGPRGSQAPLIAPLCCAAAALGMLLWSPACQAFNLDVEKL...,383.0
3,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P08648,MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDA...,17.0
4,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P05556,MNLQPIFWIGLISSVCCVFAQTDENRCLKANAKSCGECIQAGPNCG...,24.0
...,...,...,...,...,...
1779777,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P08253,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,120.0
1779778,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P14780,MSLWQPLVLVLLVLGCCFAAPRQRQSTLVLFPGDLRTNLTDRQLAE...,414.0
1779779,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P22894,MFSLKTLPFLLLLHVQISKAFPVSSKEKNTKTVQDYLEKFYQLPSN...,228.0
1779780,9999994,C1CN(CCC1CCC(=O)C2=CC3=C(C=C2)NCC3)CC4=CC(=CC=...,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,59.0


In [664]:
data_golden = rawdata.drop_duplicates().drop_duplicates(subset=("C_seq_can_smiles", "T_seq"))

In [665]:
data_golden

,C_cid,C_seq_can_smiles,T_id,T_seq,IC50
0,1000,C1=CC=C(C=C1)C(CN)O,P03470,MNPNQKIITIGSICMVVGIISLILQIGNIISIWISHSIQTGNQNHT...,870000.0
1,10000015,CC(=O)NC1CCC2=CC3=C(C(=C2C4=CC=C(C(=O)C=C14)OC...,Q6B856,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...,2100.0
2,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P53708,MSPGASRGPRGSQAPLIAPLCCAAAALGMLLWSPACQAFNLDVEKL...,383.0
3,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P08648,MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDA...,17.0
4,10000019,C1=CC=C(C=C1)C(CC(=O)O)NC(=O)CNC(=O)C2=CC(=CC=...,P05556,MNLQPIFWIGLISSVCCVFAQTDENRCLKANAKSCGECIQAGPNCG...,24.0
...,...,...,...,...,...
1779777,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P08253,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,120.0
1779778,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P14780,MSLWQPLVLVLLVLGCCFAAPRQRQSTLVLFPGDLRTNLTDRQLAE...,414.0
1779779,9999879,CC(C)C(C(=O)NO)N(CC1=CN=CC=C1)S(=O)(=O)C2=CC=C...,P22894,MFSLKTLPFLLLLHVQISKAFPVSSKEKNTKTVQDYLEKFYQLPSN...,228.0
1779780,9999994,C1CN(CCC1CCC(=O)C2=CC3=C(C=C2)NCC3)CC4=CC(=CC=...,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,59.0


In [600]:
N_filter = 7
nTperC = data_golden.groupby("C_cid").count()["T_seq"]
filtered_data = data_golden.set_index("C_cid").loc[nTperC[nTperC>=N_filter].index]

In [601]:
len_T = filtered_data.reset_index()["T_seq"].apply(lambda x: len(x))

In [602]:
T_truncated_data = filtered_data.reset_index().loc[len_T[len_T<=1024].index]

In [605]:
data = T_truncated_data

In [607]:
cid_to_seq = data[["C_cid", "C_seq_can_smiles"]].set_index("C_cid").drop_duplicates()["C_seq_can_smiles"]
i_to_C_seq, i_to_C_cid = cid_to_seq.values, cid_to_seq.index

In [609]:
tid_to_seq = data[["T_id","T_seq"]].drop_duplicates(subset=("T_seq"))
new_tids = pd.Series("", index=tid_to_seq.index)
for tid in tqdm(tid_to_seq["T_id"].unique()):
    curr_targets = tid_to_seq[tid_to_seq["T_id"]==tid]
    if len(curr_targets) > 1:
        new_tids.loc[curr_targets.index[0]] = tid
        for j in range(1,len(curr_targets)):
            new_tids.loc[curr_targets.index[j]] = tid+"-"+str(j)
    else:
        new_tids.loc[curr_targets.index[0]] = tid

100%|██████████| 2725/2725 [00:03<00:00, 889.63it/s] 


In [610]:
i_to_T_id = new_tids.values
i_to_T_seq = tid_to_seq["T_seq"].values

In [611]:
C_seq_to_i = {i_to_C_seq[i]: i for i in range(len(i_to_C_seq))}
C_cid_to_i = {i_to_C_cid[i]: i for i in range(len(i_to_C_cid))}
T_seq_to_i = {i_to_T_seq[i]: i for i in range(len(i_to_T_seq))}
T_id_to_i = {i_to_T_id[i]: i for i in range(len(i_to_T_id))}

In [612]:
new_tid_col = data["T_seq"].apply(lambda x: i_to_T_id[T_seq_to_i[x]])
data.loc[:,"T_id_unq"] = new_tid_col

In [613]:
data

,C_cid,C_seq_can_smiles,T_id,T_seq,IC50,T_id_unq
0,119,C(CC(=O)O)CN,P23978,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,2600.00,P23978
1,119,C(CC(=O)O)CN,P31649,MENRASGTTSNGETKPVCPAMEKVEEDGTLEREHWNNKMEFVLSVA...,27542.00,P31649
2,119,C(CC(=O)O)CN,P31646,MDNRVSGTTSNGETKPVCPVMEKVEEDGTLEREQWTNKMEFVLSVA...,2000.00,P31646
3,119,C(CC(=O)O)CN,Q9Z0U4,MLLLLLVPLFLRPLGAGGAQTPNATSEGCQIIHPPWEGGIRYRGLT...,30.00,Q9Z0U4
4,119,C(CC(=O)O)CN,P31650,MTAEQALPLGNGKAAEEARGSETLGGGGGGAAGTREARDKAVHERG...,11482.00,P31650
...,...,...,...,...,...,...
81489,169667776,CCC1=C(C=CC=C1S(=O)(=O)NC(CNC(=O)C2=CC=C(S2)Cl...,P08684,MALIPDLAMETWLLLAVSLVLLYLYGTHSHGLFKKLGIPGPTPLPF...,370.00,P08684
81490,169667776,CCC1=C(C=CC=C1S(=O)(=O)NC(CNC(=O)C2=CC=C(S2)Cl...,P00734,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,0.83,P00734
81491,169667776,CCC1=C(C=CC=C1S(=O)(=O)NC(CNC(=O)C2=CC=C(S2)Cl...,P00742,MGRPLHLVLLSASLAGLLLLGESLFIRREQANNILARVTRANSFLE...,0.89,P00742
81492,169667776,CCC1=C(C=CC=C1S(=O)(=O)NC(CNC(=O)C2=CC=C(S2)Cl...,P00747,MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSI...,15000.00,P00747


In [614]:
print("save?")
tosave = input()
if tosave in ["yes", "1"]:
    with open(os.path.join(data_path, "preprocessed", "i_to_C_cid.pickle"), "wb") as f:
        pickle.dump(i_to_C_cid, f)
    with open(os.path.join(data_path, "preprocessed", "i_to_T_id.pickle"), "wb") as f:
        pickle.dump(i_to_T_id, f)
    with open(os.path.join(data_path, "preprocessed", "C_cid_to_i.pickle"), "wb") as f:
        pickle.dump(C_cid_to_i, f)
    with open(os.path.join(data_path, "preprocessed", "T_id_to_i.pickle"), "wb") as f:
        pickle.dump(T_id_to_i, f)
    with open(os.path.join(data_path, "preprocessed", "i_to_C_seq.pickle"), "wb") as f:
        pickle.dump(i_to_C_seq, f)
    with open(os.path.join(data_path, "preprocessed", "i_to_T_seq.pickle"), "wb") as f:
        pickle.dump(i_to_T_seq, f)
    with open(os.path.join(data_path, "preprocessed", "C_seq_to_i.pickle"), "wb") as f:
        pickle.dump(C_seq_to_i, f)
    with open(os.path.join(data_path, "preprocessed", "T_seq_to_i.pickle"), "wb") as f:
        pickle.dump(T_seq_to_i, f)
    print("saved")
else:
    pass

save?
yes
saved


In [615]:
## 인접행렬 구축
CT_adj = np.zeros((len(i_to_C_cid), len(i_to_T_id)), dtype=np.bool)
print("Shape of C-T adjacency matrix: ", CT_adj.shape)
index_binding = data.apply(lambda x: (C_seq_to_i[x["C_seq_can_smiles"]], T_seq_to_i[x["T_seq"]]), axis=1)
CT_adj[tuple(np.array(list(zip(*index_binding.values))))] = True

Shape of C-T adjacency matrix:  (7833, 3015)


In [616]:
sps_CT_adj = sps.lil_matrix(CT_adj)

In [617]:
def chunked_spsdot(mat1, mat2, n_chunks=10, dtype=np.bool):
    if not isinstance(mat1, sps.lil.lil_matrix): mat1 = sps.lil_matrix(mat1)
    if not isinstance(mat2, sps.lil.lil_matrix): mat2 = sps.lil_matrix(mat2)
    chunk_size = int(mat1.shape[0] / n_chunks)
    sps_container = sps.csr_matrix((mat1.shape[0], mat2.shape[1]), dtype=dtype)
    print("Chunk size:",chunk_size)
    for i in tqdm(range(n_chunks)):
        if i < n_chunks-1:
            curr_mat = mat1[i*chunk_size:(i+1)*chunk_size]
            dot_res = curr_mat.dot(mat2)
            sps_container[i*chunk_size:(i+1)*chunk_size,:] = dot_res
        else:
            curr_mat = mat1[i*chunk_size:]
            dot_res = curr_mat.dot(mat2)
            sps_container[i*chunk_size:,:] = dot_res

    return sps_container

In [618]:
# sps_CTC_adj = sps_CT_adj.dot(sps_CT_adj.T)
sps_CTC_adj = chunked_spsdot(sps_CT_adj, sps_CT_adj.T, n_chunks=20)
display(sps_CTC_adj)

  0%|          | 0/20 [00:00<?, ?it/s]/home2/glee/.conda/envs/drug2/lib/python3.6/site-packages/scipy/sparse/_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
  5%|▌         | 1/20 [00:00<00:03,  5.97it/s]

Chunk size: 391


100%|██████████| 20/20 [00:05<00:00,  3.82it/s]


<7833x7833 sparse matrix of type '<class 'numpy.bool_'>'
	with 6439563 stored elements in Compressed Sparse Row format>

In [619]:
sps_CTCT_adj = sps_CTC_adj.dot(sps_CT_adj)
display(sps_CTCT_adj)

<7833x3015 sparse matrix of type '<class 'numpy.bool_'>'
	with 4955243 stored elements in Compressed Sparse Row format>

In [620]:
CTCT_adj = sps_CTCT_adj.toarray()

# Positive/Negative sampling

In [623]:
sampled_data = data.set_index("C_cid")

In [624]:
def extract_targets(data, cid):
    targets = data.loc[cid]["T_seq"]
    out = targets.values if isinstance(targets, pd.Series) else np.array([targets])
    return out

In [625]:
def split_per_C(cid, tids, split_ratio={"train": 8, "val": 1, "test": 1}):
    n_train_per_C = np.int(np.round((len(tids) / 10) * split_ratio["train"]))
    n_val_per_C = np.int(np.floor((len(tids) - (np.int(np.round((len(tids) / 10) * split_ratio["train"])))) / 2))
    n_test_per_C = len(tids) - n_train_per_C - n_val_per_C
    
    train_index = np.random.choice(np.arange(len(tids)), n_train_per_C, replace=False)
    train_remainder = np.setdiff1d(np.arange(len(tids)), train_index)
    val_index = np.random.choice(train_remainder, n_val_per_C, replace=False)
    test_index = np.setdiff1d(train_remainder, val_index)    
    
    return (tids[train_index], tids[val_index], tids[test_index])

In [626]:
train_indexes, val_indexes, test_indexes = np.empty((0,2), dtype=np.int), np.empty((0,2), dtype=np.int), np.empty((0,2), dtype=np.int)
# train_indexes, val_indexes, test_indexes = np.empty(0, dtype=np.int), np.empty(0, dtype=np.int), np.empty(0, dtype=np.int)
for i in tqdm(range(CT_adj.shape[0])): # for every C_cid
    target_indexes = np.nonzero(CT_adj[i,:])[0]
    curr_train, curr_val, curr_test = split_per_C(i, target_indexes)
    train_indexes = np.concatenate([train_indexes, np.hstack((curr_train[:,np.newaxis], np.tile(i, len(curr_train))[:,np.newaxis]))])
    val_indexes = np.concatenate([val_indexes, np.hstack((curr_val[:,np.newaxis], np.tile(i, len(curr_val))[:,np.newaxis]))])
    test_indexes = np.concatenate([test_indexes, np.hstack((curr_test[:,np.newaxis], np.tile(i, len(curr_test))[:,np.newaxis]))])

100%|██████████| 7833/7833 [00:02<00:00, 2691.22it/s]


In [628]:
print(train_indexes.shape[0]/n_whole_indexes, val_indexes.shape[0]/n_whole_indexes, test_indexes.shape[0]/n_whole_indexes)

0.8055218744879568 0.05938063247583156 0.13509749303621169


In [629]:
train_indexes

array([[   2,    0],
       [   6,    0],
       [   8,    0],
       ...,
       [ 257, 7832],
       [ 261, 7832],
       [ 236, 7832]])

In [634]:
only_CTCT = (CTCT_adj.astype(np.int8) - CT_adj.astype(np.int8)).astype(np.bool)
TC_adj = CT_adj.T
TCTC_adj = CTCT_adj.T
only_TCTC = only_CTCT.T

In [635]:
posneg_cols_index = ["Target_index", "Anchor_index", "Positive_index", "Negative_index"]

In [636]:
tr = pd.DataFrame(train_indexes, columns=posneg_cols_index[:2]).sort_values("Target_index").set_index("Target_index")

In [637]:
tr

,Anchor_index
Target_index,
0,1605
0,0
0,6445
0,904
0,139
...,...
3011,7625
3012,7623
3013,7663


In [643]:
train_container = pd.DataFrame(columns=posneg_cols_index[2:])
for curr_index in tqdm(tr.index.unique()):
    n_anchors_per_T = tr.loc[curr_index].shape[0]
    pos_per_T = np.nonzero(TCTC_adj[curr_index,:])[0]
    neg_per_T = np.nonzero(TCTC_adj[curr_index,:]==0)[0]
    curr_pos = np.random.choice(pos_per_T, n_anchors_per_T)
    curr_neg = np.random.choice(neg_per_T, n_anchors_per_T)    
    curr_posneg = pd.DataFrame(np.hstack((curr_pos[:,np.newaxis], curr_neg[:,np.newaxis])), index=np.tile(curr_index, n_anchors_per_T), columns=posneg_cols_index[2:])
    train_container = pd.concat([train_container, curr_posneg], axis=0)

100%|██████████| 2820/2820 [00:06<00:00, 438.36it/s]


In [645]:
tr.loc[:,posneg_cols_index[2:]] = train_container

In [646]:
tr = tr.reset_index()

In [648]:
train_set = pd.DataFrame(index=tr.index)
train_set.loc[:,"T_seq"] = tr["Target_index"].apply(lambda x: i_to_T_seq[x])
train_set.loc[:,"C_anc"] = tr["Anchor_index"].apply(lambda x: i_to_C_seq[x])
train_set.loc[:,"C_pos"] = tr["Positive_index"].apply(lambda x: i_to_C_seq[x])
train_set.loc[:,"C_neg"] = tr["Negative_index"].apply(lambda x: i_to_C_seq[x])

In [650]:
va = pd.DataFrame(val_indexes, columns=posneg_cols_index[:2]).sort_values("Target_index")

In [651]:
va

,Target_index,Anchor_index
999,0,1862
281,0,443
94,1,139
519,2,904
887,4,1605
...,...,...
3322,2966,7117
3408,2982,7302
3472,2992,7417
3475,2993,7429


In [652]:
val_set = pd.DataFrame(index=va.index)
val_set.loc[:,"T_seq"] = va["Target_index"].apply(lambda x: i_to_T_seq[x])
val_set.loc[:,"C_anc"] = va["Anchor_index"].apply(lambda x: i_to_C_seq[x])

In [653]:
te = pd.DataFrame(test_indexes, columns=posneg_cols_index[:2]).sort_values("Target_index")

In [654]:
te

,Target_index,Anchor_index
2682,1,2404
1813,1,1605
2073,4,1862
527,5,443
173,10,139
...,...,...
7682,2982,7264
7861,2994,7429
7870,3001,7438
8039,3009,7601


In [655]:
test_set = pd.DataFrame(index=te.index)
test_set.loc[:,"T_seq"] = te["Target_index"].apply(lambda x: i_to_T_seq[x])
test_set.loc[:,"C_anc"] = te["Anchor_index"].apply(lambda x: i_to_C_seq[x])

In [656]:
data_path

'/home2/glee/Drug_Discovery_Research/data'

In [657]:
train_set.to_csv(data_path+"/splitted/train.csv")

In [658]:
val_set.to_csv(data_path+"/splitted/val.csv")

In [659]:
test_set.to_csv(data_path+"/splitted/test.csv")

In [501]:
test_set

,T_seq,C_anc
264,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,CNCCC=C1C2=CC=CC=C2CCC3=CC=CC=C31
527,MDNRVSGTTSNGETKPVCPVMEKVEEDGTLEREQWTNKMEFVLSVA...,CC1=C(SC=C1)C(=CCCN2CCCC(C2)C(=O)O)C3=C(C=CS3)C
2074,MDNRVSGTTSNGETKPVCPVMEKVEEDGTLEREQWTNKMEFVLSVA...,COC1=CC=C(C=C1)C(C2=CC=C(C=C2)OC)(C3=CC=C(C=C3...
1814,MATNGSKVADGQISTEVSEAPVANDKPKTLVVKVQKKAADLPDRDT...,COC1=CC=CC=C1C2(CCN(CC2)CCCN3C4=CC=CC=C4C5=CC=...
0,MGSGKVFLFSPSLLWSQTRGVRLIFLLLTLHLGNCIDKADDEDDED...,C(CC(=O)O)CN
...,...,...
7726,MSTEGGGRRCQAQVSRRISFSASHRLYSKFLSDEENLKLFGKCNNP...,C1=CC(=CC=C1C#CC2=C(C=C(C=C2)C(F)(F)F)Cl)NC(=O...
7796,MPFSVSWGVLLLAGLCCLVPSSLVEDPQGDAAQKTDTSHHDQGDWE...,C1COCCC(=C1)C2=NN=C3N2C4=C(C=CC(=C4)C5=CC(=CN=...
7862,MALDCLLLFLLASAVAAMEETLMDTRTATAELGWTANPASGWEEVS...,CCNC(=O)OC1CCC2(C(C1)CCC3C2CCC4(C3CCC4C(C)CCC(...
8233,MGPSCPVFLSFTKLSLWWLLLTPAGGEEAKRPPPRAPGDPLSSPSP...,CC(C1C(=O)NC(C(=O)N2CCCC2C(=O)NC(C(=O)NC3CSSCC...


In [13]:
index_train, index_test = train_test_split(C_cids, shuffle=True, test_size=0.2)

In [14]:
data_TRAIN = data.loc[index_train]
data_TEST = data.loc[index_test]

In [15]:
print("#TRAIN SET: {}, #TEST SET: {}".format(len(data_TRAIN), len(data_TEST)))

#TRAIN SET: 1039759, #TEST SET: 259523


## Save function for sparse matrix

In [24]:
def save_hdf5(data_to_save, data_path):
    import h5py
    from scipy.sparse import csr_matrix
    with h5py.File(data_path, "w") as f:
        if isinstance(data_to_save, csr_matrix):
            f.create_dataset("data", data=data_to_save.data)
            f.create_dataset("indices", data=data_to_save.indices)
            f.create_dataset("indptr", data=data_to_save.indptr)
            f.attrs["shape"] = data_to_save.shape
            f.attrs["type"] = "sparse"
        else:
            f.create_dataset("data", data=data_to_save)
            f.attrs["type"] = "dense"
    
def load_hdf5(data_path):
    import h5py
    from scipy.sparse import csr_matrix
    with h5py.File(data_path, "r") as f:
        if f.attrs["type"]=="sparse":
            data = f["data"][:]
            indices = f["indices"][:]
            indptr = f["indptr"][:]
            shape = f.attrs["shape"]
            return csr_matrix((data, indices, indptr), shape=shape)
        else:
            return f["data"][:]

In [25]:
save_hdf5(TRAIN_sps_CT_adj, os.path.join(data_path, "preprocessed", "TRAIN_sps_CT_adj.h5"))
save_hdf5(TRAIN_sps_CTCT_adj, os.path.join(data_path, "preprocessed", "TRAIN_sps_CTCT_adj.h5"))
save_hdf5(TEST_sps_CT_adj, os.path.join(data_path, "preprocessed", "TEST_sps_CT_adj.h5"))
save_hdf5(TEST_sps_CTCT_adj, os.path.join(data_path, "preprocessed", "TEST_sps_CTCT_adj.h5"))

In [26]:
!pip install levenshtein

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 171 kB 795 kB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
loaded_sps_CT_adj = load_hdf5(os.path.join(data_path, "preprocessed", "TRAIN_sps_CTCT_adj.h5"))

In [ ]:
loaded_sps_CT_adj

## C-C, T-T relationships based on Levenshtein distance 

In [226]:
import re

In [240]:
C_samples = C_samples.astype(np.str)

In [242]:
C_samples

array(['B(C#N)(C1=CC(=CC=C1)C=CC(=O)N2C3=CC=CC=C3N=N2)O',
       'B(C#N)(C1=CC=C(C=C1)C=CC(=O)N2C3=CC=CC=C3N=N2)O',
       'B(C(=C)CNC1=C(C2=C(C(=C1)C3=CC=C(C=C3)C(C)C)OCC2)C#N)(O)O', ...,
       'C1=C(N(C(=C1)O)C2=CC(=C(C(=C2)Cl)Cl)Cl)O',
       'C1=C(N(C(=S)S1)CC(=O)O)O', 'C1=C(N(C(=S)S1)CCCCCC(=O)O)O'],
      dtype='<U510')

In [274]:
Cs_cleaned = list(map(lambda x: list(x), index_to_C))

In [275]:
vocab_C = np.unique(np.concatenate(Cs_cleaned))

In [298]:
ddd = {"<PAD>": 0}
ddd.update({"A": 1})

In [304]:
token_to_index = {"<PAD>": 0}
token_to_index.update({vocab_C[i]: i+1 for i in range(len(vocab_C))})
index_to_token = {0: "<PAD>"}
index_to_token.update({i+1: vocab_C[i] for i in range(len(vocab_C))})

In [305]:
token_to_index

{'<PAD>': 0,
 '#': 1,
 '%': 2,
 '(': 3,
 ')': 4,
 '+': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 '=': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'F': 22,
 'G': 23,
 'H': 24,
 'I': 25,
 'K': 26,
 'L': 27,
 'M': 28,
 'N': 29,
 'O': 30,
 'P': 31,
 'R': 32,
 'S': 33,
 'T': 34,
 'V': 35,
 'W': 36,
 'Z': 37,
 '[': 38,
 ']': 39,
 'a': 40,
 'b': 41,
 'c': 42,
 'd': 43,
 'e': 44,
 'g': 45,
 'i': 46,
 'l': 47,
 'n': 48,
 'o': 49,
 'r': 50,
 's': 51,
 't': 52,
 'u': 53}

In [296]:
Cs_vectorized = [[token_to_index[token] for token in s] for s in Cs_cleaned]

In [306]:
max_len_C = 100

In [320]:
Cs_vectorized[0] + list(np.tile(token_to_index["<PAD>"], max_len_C - len(Cs_vectorized[0])))

100

In [322]:
temp = Cs_vectorized[0][:max_len_C] if len(Cs_vectorized[0]) > max_len_C else Cs_vectorized[0] + list(np.tile(token_to_index["<PAD>"], max_len_C - len(Cs_vectorized[0])))

In [324]:
len(temp)

100

In [325]:
Cs_padded = np.array([s[:max_len_C] if len(s)>max_len_C else s + list(np.tile(token_to_index["<PAD>"], max_len_C - len(s))) for s in Cs_vectorized])

In [332]:
def cal_sim(s1, s2):
    return 1 - lvs.hamming(s1, s2) / max_len_C

In [334]:
cal_sim(Cs_padded[0], Cs_padded[100])

0.5700000000000001

In [335]:
Cs_padded

array([[19,  2, 20, ...,  0,  0,  0],
       [19,  2, 20, ...,  0,  0,  0],
       [19,  2, 20, ...,  0,  0,  0],
       ...,
       [37, 29,  5, ...,  0,  0,  0],
       [37, 29,  5, ...,  0,  0,  0],
       [37, 36, 47, ...,  0,  0,  0]])

In [336]:
from scipy.spatial import KDTree

In [362]:
kdtree = KDTree(Cs_padded[:10000])

In [363]:
kdtree

In [339]:
def find_similar_strings(tree, X, threshold):
    similar_pairs = []
    for i in range(len(X)):
        distances, indices = tree.query(X[i], k=len(X), distance_upper_bound=threshold)
        for j in indices:
            if j != i and j < len(X) and distances[j] < threshold:
                similar_pairs.append((i, j))
    return similar_pairs

In [370]:
distances, indices = kdtree.query(Cs_padded[100], k=len(Cs_padded[:10000]), distance_upper_bound=1)

In [371]:
distances

array([ 0., inf, inf, ..., inf, inf, inf])

In [360]:
kdtree.query_ball_point(Cs_padded[0], 10)

[0]

In [343]:
sim_pairs = find_similar_strings(kdtree, Cs_padded[:100], 0.3)

In [344]:
sim_pairs

[]

In [50]:
s1 = index_to_C[0]
s2 = index_to_C[3000]
display(s1, s2)

'B(C#N)(C1=CC(=CC=C1)C=CC(=O)N2C3=CC=CC=C3N=N2)O'

'C#CCOC(=O)C1=CC(=C(C=C1)O)NC(=O)COC2=CC=CC=C2C3(N=N3)C(F)(F)F'

In [55]:
print("lvs distance: {}".format(lvs.distance(s1, s2)))
print("lvs rel-distance: {}".format(lvs.ratio(s1, s2)))

lvs distance: 27
lvs rel-distance: 0.6481481481481481


In [178]:
lvs_threshold = 0.75

In [211]:
C_samples = index_to_C[:100]
n_Cs = C_samples.shape[0]

In [180]:
CC_adj = np.zeros((n_Cs, n_Cs), dtype=np.bool)
for i in tqdm(range(n_Cs)):
    CC_adj[i,i] = True
    similarities = list(map(lambda s1,s2: lvs.ratio(s1, s2) > lvs_threshold, np.tile(C_samples[0], n_Cs-(i+1)), C_samples[i+1:]))
    CC_adj[i,i+1:] = similarities
    CC_adj[i+1:,i] = similarities

100%|██████████| 1000/1000 [00:00<00:00, 1023.10it/s]


In [208]:
import numpy as np
from numba import jit
from joblib import Parallel, delayed
import itertools
from scipy.sparse import lil_matrix

# Levenshtein 거리 계산 함수
@jit(nopython=True)
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = np.arange(len(s2) + 1, dtype=np.int32)
    for i, c1 in enumerate(s1):
        current_row = np.zeros(len(s2) + 1, dtype=np.int32)
        current_row[0] = i + 1
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row[j + 1] = min(insertions, deletions, substitutions)
        previous_row = current_row

    return previous_row[-1]

# 임계값 설정
THRESHOLD = 10  # 예시 임계값, 필요에 따라 조정

# 거리 계산 함수를 병렬로 적용
def calculate_similarity(i, j):
    sim = lvs.ratio(C_samples[i], C_samples[j])
    return (i, j, sim >= lvs_threshold)

# 인접 행렬의 크기 설정
# n = len(strings)
# adj_matrix = lil_matrix((n, n), dtype=bool)

# 병렬 처리
# results = Parallel(n_jobs=-1)(delayed(calculate_distance)(i, j) for i, j in itertools.combinations(range(n), 2))

In [206]:
C_samples.shape

(100,)

In [223]:
C_samples = index_to_C[:5000]
n_Cs = C_samples.shape[0]

In [224]:
res = Parallel(n_jobs=-1)(delayed(calculate_similarity)(i, j) for i, j in itertools.combinations(range(n_Cs), 2))

KeyboardInterrupt: 

In [215]:
res

[(0, 1, True),
 (0, 2, False),
 (0, 3, False),
 (0, 4, False),
 (0, 5, False),
 (0, 6, False),
 (0, 7, False),
 (0, 8, False),
 (0, 9, False),
 (0, 10, False),
 (0, 11, False),
 (0, 12, False),
 (0, 13, False),
 (0, 14, False),
 (0, 15, False),
 (0, 16, False),
 (0, 17, False),
 (0, 18, False),
 (0, 19, False),
 (0, 20, False),
 (0, 21, False),
 (0, 22, False),
 (0, 23, False),
 (0, 24, False),
 (0, 25, False),
 (0, 26, False),
 (0, 27, False),
 (0, 28, False),
 (0, 29, False),
 (0, 30, False),
 (0, 31, False),
 (0, 32, False),
 (0, 33, False),
 (0, 34, False),
 (0, 35, False),
 (0, 36, False),
 (0, 37, False),
 (0, 38, False),
 (0, 39, False),
 (0, 40, False),
 (0, 41, False),
 (0, 42, False),
 (0, 43, False),
 (0, 44, False),
 (0, 45, False),
 (0, 46, False),
 (0, 47, False),
 (0, 48, False),
 (0, 49, False),
 (0, 50, False),
 (0, 51, False),
 (0, 52, False),
 (0, 53, False),
 (0, 54, False),
 (0, 55, False),
 (0, 56, False),
 (0, 57, False),
 (0, 58, False),
 (0, 59, False),
 (0, 60

In [218]:
spsres = lil_matrix((n_Cs, n_Cs), dtype=np.bool)

In [219]:
for i, j, is_close in res:
    if is_close:
        spsres[i,j] = True
        spsres[j,i] = True

In [222]:
spsres.toarray()

array([[False,  True, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False,  True,  True],
       [False, False, False, ...,  True, False,  True],
       [False, False, False, ...,  True,  True, False]])

## END